In [192]:
import numpy as np 
import pandas as pd
from statsmodels import api as sm
from statsmodels.stats.sandwich_covariance import cov_hac
from scipy import stats

In [193]:
def analyze_double_sort_results(results):
    """
    返回一个整合后的结果表格，格式如下：
        Portfolio   CGO1     CGO3     CGO5
        P1          0.0102   0.0065   0.0035
        P3          0.0080   0.0081   0.0098
        P5          0.0097   0.0046   0.0068
        P5 - P1    -0.0005  -0.0019   0.0033
        t-stat      (-0.12)  (-0.74)  (0.86)

    Parameters:
        results: DataFrame with ['date', 'cgo_group', 'proxy_group', 'portfolio_return']
    
    Returns:
        pd.DataFrame: 整合后的结果表
    """
    # 生成 5x5 平均收益表（原始）
    mean_table = results.pivot_table(
        index='proxy_group',
        columns='cgo_group',
        values='portfolio_return',
        aggfunc='mean'
    )

    # 转宽表
    wide = results.pivot_table(
        index='date',
        columns=['cgo_group', 'proxy_group'],
        values='portfolio_return'
    )

    # 计算每个 cgo_group 的 P5 - P1 和 Newey-West t-stat
    diffs = []
    t_stats = []

    for cgo in [0, 2, 4]:  # 只取 CGO1, CGO3, CGO5 (对应 cgo_group 0,2,4)
        p5_series = wide[(cgo, 4)]
        p1_series = wide[(cgo, 0)]

        diff = p5_series - p1_series
        diff_clean = diff.dropna()

        if len(diff_clean) < 2:
            diffs.append(np.nan)
            t_stats.append(np.nan)
        else:
            mean_diff = diff_clean.mean()
            lag = 6
            X = np.ones((len(diff_clean), 1))
            model = sm.OLS(diff_clean, X).fit(cov_type='HAC', cov_kwds={'maxlags': lag})
            t_stat = model.tvalues.iloc[0]
            diffs.append(mean_diff)
            t_stats.append(t_stat)

    # 构建最终表格
    # 行：P1, P3, P5, P5-P1, t-stat
    # 列：CGO1, CGO3, CGO5 (对应 cgo_group 0,2,4)
    data = {
        'CGO1': [
            mean_table.loc[0, 0],  # P1
            mean_table.loc[2, 0],  # P3
            mean_table.loc[4, 0],  # P5
            diffs[0],              # P5-P1
            f"({t_stats[0]:.2f})" if not np.isnan(t_stats[0]) else "NaN"  # t-stat
        ],
        'CGO3': [
            mean_table.loc[0, 2],
            mean_table.loc[2, 2],
            mean_table.loc[4, 2],
            diffs[1],
            f"({t_stats[1]:.2f})" if not np.isnan(t_stats[1]) else "NaN"
        ],
        'CGO5': [
            mean_table.loc[0, 4],
            mean_table.loc[2, 4],
            mean_table.loc[4, 4],
            diffs[2],
            f"({t_stats[2]:.2f})" if not np.isnan(t_stats[2]) else "NaN"
        ]
    }

    index = ['P1', 'P3', 'P5', 'P5 - P1', 't-stat']
    result_df = pd.DataFrame(data, index=index)

    return result_df


In [194]:
def dependent_double_sort_by_rank(df, sort_var1, sort_var2, n1=5, n2=5):
    """
    Dependent Double Sorting using ranking + qcut on ranks.
    
    Steps:
    1. For each date, rank sort_var1 (method='first'), then qcut(rank, n1) → group1
    2. Within each (date, group1), rank sort_var2 (method='first'), then qcut(rank, n2) → group2
    3. Compute equal-weighted portfolio return for each (date, group1, group2)
    
    Parameters:
    - df: DataFrame with ['date', 'asset', sort_var1, sort_var2, 'future_return']
    - sort_var1: str, control variable (e.g., 'cgo')
    - sort_var2: str, main variable (e.g., 'risk')
    - n1, n2: int, number of groups (default 5)
    
    Returns:
    - DataFrame: ['date', 'group1', 'group2', 'portfolio_return']
    """
    df = df.copy()
    df = df.dropna(subset=[sort_var1, sort_var2, 'future_return'])
    
    # Rank and group by sort_var1 (control variable) within each date
    df['rank1'] = df.groupby('date')[sort_var1].rank(method='first')
    df['cgo_group'] = df.groupby('date')['rank1'].transform(
        lambda x: pd.qcut(x, n1, labels=False, duplicates='raise')
    )
    
    # Within each (date, group1), rank and group by sort_var2
    df['rank2'] = df.groupby(['date', 'cgo_group'])[sort_var2].rank(method='first')
    df['proxy_group'] = df.groupby(['date', 'cgo_group'])['rank2'].transform(
        lambda x: pd.qcut(x, n2, labels=False, duplicates='raise')
    )
    
    # Compute portfolio returns (equal-weighted)
    portfolio_ret = df.groupby(['date', 'cgo_group', 'proxy_group'])['future_return'].mean().reset_index()
    portfolio_ret.rename(columns={'future_return': 'portfolio_return'}, inplace=True)
    
    return portfolio_ret


___

In [195]:
data_panel = pd.read_pickle('./Data_Submission/final_data_panel_proxies_window30month.pkl')

data_panel

,date,asset,beta_MKT,ivol,retvol,age,disp,CFVOL,LOGBM,LOGME,mom_minus1_0,mom_minus12_minus1,mom_minus36_minus12,mom_minus11_minus2,cgo,turnover,future_return,score
0,2014-01,000006,0.714892,-0.385213,-0.253707,-0.698990,NaN,-0.195242,0.852785,-0.290561,-1.010538,-0.434441,1.405009,0.228706,-0.437893,-0.381935,-0.016018,-1.735858
1,2014-01,000021,0.902618,-0.210387,0.403084,-0.672945,NaN,0.330226,0.302500,0.840230,2.201666,0.015993,-1.109247,0.205026,0.858371,-0.227697,-0.077961,-0.956005
2,2014-01,000028,-0.789488,0.177394,0.003037,-0.682054,NaN,-0.804418,-1.637611,1.817995,1.169076,0.485133,1.219088,0.203879,2.514612,-0.564752,-0.078858,1.029076
3,2014-01,000030,-0.926812,3.139430,3.924975,-0.680567,NaN,0.162950,0.281369,0.279086,-0.336719,-1.182369,0.487879,-1.298161,-0.311534,-0.033620,0.255537,1.454450
4,2014-01,000031,0.503439,-0.779173,-0.700685,-0.679068,NaN,-0.857062,0.960635,-0.096190,-0.409915,-0.677725,-0.352625,-0.694764,-0.952216,-0.911117,-0.090141,-0.388839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57398,2023-12,688772,NaN,NaN,1.188984,2.479040,1.459955,-0.483142,-0.825818,0.268728,1.004084,0.500160,NaN,-0.035467,1.150507,0.641422,-0.390277,0.647214
57399,2023-12,688778,NaN,NaN,0.525038,2.227011,0.561135,0.007475,0.054129,-0.952300,0.288576,-0.910079,NaN,-0.952374,-0.640407,-0.292542,-0.240666,0.714458
57400,2023-12,688779,NaN,NaN,-0.225561,2.227011,1.209956,0.853260,0.108926,-1.478866,0.033653,-1.592488,NaN,-1.541132,-2.424496,0.662592,-0.230137,0.961017
57401,2023-12,688819,-0.840286,NaN,0.298146,1.578404,0.809786,0.458108,0.156839,0.563051,-0.531444,-0.587884,NaN,-0.364451,-0.494163,0.508415,-0.116046,0.198923


In [196]:
data_panel.isna().sum()

date                      0
asset                     0
beta_MKT               2319
ivol                   7541
retvol                    0
age                       0
disp                   2502
CFVOL                  2706
LOGBM                     7
LOGME                     1
mom_minus1_0              0
mom_minus12_minus1        0
mom_minus36_minus12    3272
mom_minus11_minus2        0
cgo                     273
turnover                  3
future_return             0
score                     7
dtype: int64

___

## Double Sort Age

In [197]:
results = dependent_double_sort_by_rank(
    df=data_panel,
    sort_var1='cgo',
    sort_var2='age',
    n1=5,
    n2=5
)

results


,date,cgo_group,proxy_group,portfolio_return
0,2014-01,0,0,0.042841
1,2014-01,0,1,0.041140
2,2014-01,0,2,0.027926
3,2014-01,0,3,0.008623
4,2014-01,0,4,0.009909
...,...,...,...,...
2995,2023-12,4,0,-0.037106
2996,2023-12,4,1,-0.037166
2997,2023-12,4,2,-0.104788
2998,2023-12,4,3,-0.129667


In [198]:
res_age = analyze_double_sort_results(results)
res_age

,CGO1,CGO3,CGO5
P1,0.0073,0.009383,0.004595
P3,0.006723,0.009448,0.003208
P5,0.006952,0.006301,0.008871
P5 - P1,-0.000348,-0.003082,0.004276
t-stat,(-0.10),(-1.14),(1.15)


___

## Double Sort DISPER

In [199]:
results = dependent_double_sort_by_rank(
    df=data_panel,
    sort_var1='cgo',
    sort_var2='disp',
    n1=5,
    n2=5
)

results

,date,cgo_group,proxy_group,portfolio_return
0,2014-01,0,0,0.044709
1,2014-01,0,1,0.056443
2,2014-01,0,2,-0.013429
3,2014-01,0,3,-0.034057
4,2014-01,0,4,0.031552
...,...,...,...,...
2995,2023-12,4,0,-0.052988
2996,2023-12,4,1,-0.068848
2997,2023-12,4,2,-0.101076
2998,2023-12,4,3,-0.085769


In [200]:
res_disper = analyze_double_sort_results(results)
res_disper

,CGO1,CGO3,CGO5
P1,0.005634,0.005606,0.005134
P3,0.007028,0.008221,0.008274
P5,0.008379,0.008965,0.009805
P5 - P1,0.002745,0.003359,0.004671
t-stat,(1.35),(1.28),(0.90)


___

## Double Sort Beta

In [201]:
results = dependent_double_sort_by_rank(
    df=data_panel,
    sort_var1='cgo',
    sort_var2='beta_MKT',
    n1=5,
    n2=5
)

results

,date,cgo_group,proxy_group,portfolio_return
0,2014-01,0,0,0.044808
1,2014-01,0,1,0.015984
2,2014-01,0,2,0.051107
3,2014-01,0,3,0.007334
4,2014-01,0,4,0.007967
...,...,...,...,...
2995,2023-12,4,0,-0.059584
2996,2023-12,4,1,-0.037280
2997,2023-12,4,2,-0.094037
2998,2023-12,4,3,-0.075409


In [202]:
res_beta = analyze_double_sort_results(results)
res_beta

,CGO1,CGO3,CGO5
P1,0.006798,0.010549,0.007968
P3,0.008321,0.007626,0.008281
P5,0.003101,0.006026,0.006451
P5 - P1,-0.003696,-0.004523,-0.001517
t-stat,(-1.03),(-1.33),(-0.35)


___

## Double Sort IVOL

In [203]:
results = dependent_double_sort_by_rank(
    df=data_panel,
    sort_var1='cgo',
    sort_var2='ivol',
    n1=5,
    n2=5
)

results

,date,cgo_group,proxy_group,portfolio_return
0,2014-01,0,0,0.013513
1,2014-01,0,1,0.048810
2,2014-01,0,2,0.030241
3,2014-01,0,3,0.024206
4,2014-01,0,4,0.036468
...,...,...,...,...
2995,2023-12,4,0,0.009419
2996,2023-12,4,1,-0.102538
2997,2023-12,4,2,-0.095074
2998,2023-12,4,3,-0.100903


In [204]:
res_ivol = analyze_double_sort_results(results)
res_ivol

,CGO1,CGO3,CGO5
P1,0.010331,0.015667,0.012571
P3,0.003948,0.006994,0.009316
P5,0.005868,0.002332,0.001753
P5 - P1,-0.004463,-0.013335,-0.010818
t-stat,(-1.06),(-3.61),(-2.48)


___

## Double Sort RETVOL

In [205]:
results = dependent_double_sort_by_rank(
    df=data_panel,
    sort_var1='cgo',
    sort_var2='retvol',
    n1=5,
    n2=5
)

results

,date,cgo_group,proxy_group,portfolio_return
0,2014-01,0,0,0.031960
1,2014-01,0,1,0.036022
2,2014-01,0,2,0.014151
3,2014-01,0,3,0.035386
4,2014-01,0,4,0.014001
...,...,...,...,...
2995,2023-12,4,0,-0.010194
2996,2023-12,4,1,-0.098202
2997,2023-12,4,2,-0.085609
2998,2023-12,4,3,-0.126702


In [206]:
res_retvol = analyze_double_sort_results(results)
res_retvol

,CGO1,CGO3,CGO5
P1,0.011388,0.015304,0.012622
P3,0.005098,0.006922,0.005658
P5,0.00459,0.002665,0.004996
P5 - P1,-0.006798,-0.012639,-0.007626
t-stat,(-1.74),(-3.23),(-1.52)


___

## Double Sort CFVOL

In [207]:
results = dependent_double_sort_by_rank(
    df=data_panel,
    sort_var1='cgo',
    sort_var2='CFVOL',
    n1=5,
    n2=5
)

results

,date,cgo_group,proxy_group,portfolio_return
0,2014-01,0,0,0.053334
1,2014-01,0,1,0.006216
2,2014-01,0,2,0.016016
3,2014-01,0,3,0.017328
4,2014-01,0,4,0.032176
...,...,...,...,...
2995,2023-12,4,0,-0.070829
2996,2023-12,4,1,-0.117254
2997,2023-12,4,2,-0.035476
2998,2023-12,4,3,-0.137693


In [208]:
res_cfvol = analyze_double_sort_results(results)
res_cfvol

,CGO1,CGO3,CGO5
P1,0.006965,0.009778,0.006394
P3,0.007197,0.010414,0.011955
P5,0.00602,0.006578,0.00531
P5 - P1,-0.000945,-0.0032,-0.001084
t-stat,(-0.34),(-0.74),(-0.26)


___

## 结果汇总

In [209]:
from IPython.display import display, HTML

def display_multi_panel(proxy_dfs, proxy_names):
    """
    在 Jupyter 中显示多面板表格，每面板对应一个 Proxy。
    """
    html_parts = []
    
    for i, (df, name) in enumerate(zip(proxy_dfs, proxy_names)):
        # 给每个面板加标题
        title = f"<h4>Proxy = {name}</h4>"
        
        # 转换为 HTML 表格
        table_html = df.to_html(classes='table table-striped', border=0)
        
        # 包装进 div
        panel_html = f"""
        <div style="display:inline-block; vertical-align:top; margin:10px; padding:10px; border:1px solid #ccc;">
            {title}
            {table_html}
        </div>
        """
        html_parts.append(panel_html)
    
    # 2列布局
    html_content = "<div style='display:flex; flex-wrap:wrap;'>"
    html_content += "".join(html_parts)
    html_content += "</div>"
    
    display(HTML(html_content))

display_multi_panel(
    [res_age, res_disper, res_beta, res_ivol, res_retvol, res_cfvol],
    ['β', 'DISPER', 'RETVOL', 'IVOL', 'CFVOL', '1/AGE']
)

,CGO1,CGO3,CGO5
P1,0.0073,0.009383,0.004595
P3,0.006723,0.009448,0.003208
P5,0.006952,0.006301,0.008871
P5 - P1,-0.000348,-0.003082,0.004276
t-stat,(-0.10),(-1.14),(1.15)
,CGO1,CGO3,CGO5
P1,0.005634,0.005606,0.005134
P3,0.007028,0.008221,0.008274
P5,0.008379,0.008965,0.009805
P5 - P1,0.002745,0.003359,0.004671


___